# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

VAE Training Notes:

* Higher KL weight results in network focusing less on minimizing reconstruction loss and thus produces lower quality reconstructions
* KL weight < .1 sees diminishing gains; use kl == .1
* Reconstruction loss is very important
* Latent dim of 100 borrowed from reference paper
* Latent dim of 1000 results in lower losses but overtrains (also, reconstructions do not look as good)
* TODO: experiment with latent dim of 350
* Learning rate of 0.0001 borrowed from reference papers; 0.001 trains faster but hits nan loss after a while

VAEGAN Training Notes:

* Architecture as-is (after adjusting loss functions from GAN loss to VAEGAN loss)

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 128,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,  # 0.001 hits nan loss eventually
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 0.8,
                "kl_div_loss_weight": .1,
                "recon_loss_weight": 10000,
                "ll_weight": 1,
                "dec_weight": 100,
                "latent_dim": 1000,
                "verbose": True,
                "debug": True,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 5,
                "save_step": 5,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                #'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                #'tb_compare': [('vaegan_logless_loss', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-22_16-01-54/tb')],
                #'tb_compare': [('voxel_vae_toilet_200epochs_1', os.path.join(MODEL_DIR, 'voxel_vae_toilet_200epochs_1', 'tb'))],
                'tb_compare': [('vae_200epochs_p1kl', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-25_11-05-00/tb')],
                'no_gan': True,
                'monitor_memory': True,
                # these settings control how often the components' optimizers are executed during the training loop
                'train_vae_cadence': 1,
                'train_gan_cadence': 1,
                'dis_noise': 0,
                'adaptive_lr': False
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b''


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-26_05-10-08
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.adaptive_lr"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.dis_noise"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNI

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-26_05-10__root.log
Starting train_vaegan main
Numpy random seed: 30304272
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-26_05-10-08/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 55.50
Initializing VoxelVaegan
encoder/enc_conv1/batchnorm/add_1:0: (?, 30, 30, 30, 8)
encoder/batch_normalization/batchnorm/add_1:0: (?, 15, 15, 15, 16)
encoder/batch_normalization_1/batchnorm/add_1:0: (?, 13, 13, 13, 32)
encoder/batch_normalization_2/batchnorm/add_1:0: (?, 7, 7, 7, 64)
encoder/batch_normalization_3/batchnorm/add_1:0: (?, 7, 7, 7, 343)
encoder/dense_1/BiasAdd:0: (?, 1000)
encoder/dense_2/BiasAdd:0: (?, 1000)
epsilon: (?, 1000)
z: (?, 1000)
input_z: (?, 1000)
decoder/dec_dense1/Bias

<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_2/moving_variance/Adam_1:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/kernel/Adam_1:0' shape=(64, 343) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/dense/bias/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/gamma/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/beta/Adam_1:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normalization_3/moving_mean/Adam:0' shape=(343,) dtype=float32_ref>
<tf.Variable 'encoder/batch_normaliz

<tf.Variable 'decoder/dec_conv4/kernel/Adam_1:0' shape=(4, 4, 4, 8, 16) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/gamma/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/beta/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_mean/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/batch_normalization_4/moving_variance/Adam_1:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam:0' shape=(3, 3, 3, 1, 8) dtype=float32_ref>
<tf.Variable 'decoder/dec_conv5/kernel/Adam_1:0' shape=(3, 3, 3, 1, 8) dtype=fl

Memory Use (GB): 3.6068153381347656
Epoch: 0 / 201, Batch: 28 (0 / 3712), Elapsed time: 2.00 mins
Enc Loss = 622.41, KL Divergence = 789.08, Reconstruction Loss = 0.05, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 2.07 mins
Memory Use (GB): 3.7132415771484375
Epoch: 0 / 201, Batch: 29 (0 / 3840), Elapsed time: 2.07 mins
Enc Loss = 582.20, KL Divergence = 800.32, Reconstruction Loss = 0.05, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 2.14 mins
Memory Use (GB): 3.6855697631835938
Epoch: 0 / 201, Batch: 30 (0 / 3968), Elapsed time: 2.14 mins
Enc Loss = 578.25, KL Divergence = 649.29, Reconstruction Loss = 0.05, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 2.21 mins
Memory Use (GB): 3.646533966064453
Epoch: 0 / 201, Batch: 31 (0 / 4096), Elapsed time: 2.21 mins
Enc Loss = 540.17, KL Divergence = 657.14, Reconstruction Loss = 0.05, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 2.2

...

Memory Use (GB): 3.8204612731933594
Epoch: 69 / 201, Batch: 37 (4864 / 5504), Elapsed time: 216.58 mins
Enc Loss = 110.55, KL Divergence = 261.90, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 216.65 mins
Memory Use (GB): 3.790843963623047
Epoch: 69 / 201, Batch: 38 (4992 / 5504), Elapsed time: 216.65 mins
Enc Loss = 120.91, KL Divergence = 267.32, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 216.72 mins
Memory Use (GB): 3.7492713928222656
Epoch: 69 / 201, Batch: 39 (5120 / 5504), Elapsed time: 216.72 mins
Enc Loss = 120.03, KL Divergence = 286.33, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 216.79 mins
Memory Use (GB): 3.664794921875
Epoch: 69 / 201, Batch: 40 (5248 / 5504), Elapsed time: 216.79 mins
Enc Loss = 116.00, KL Divergence = 276.25, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = 

Memory Use (GB): 3.660919189453125
Epoch: 70 / 201, Batch: 17 (2304 / 5504), Elapsed time: 218.46 mins
Enc Loss = 115.28, KL Divergence = 262.51, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 218.53 mins
Memory Use (GB): 3.605541229248047
Epoch: 70 / 201, Batch: 18 (2432 / 5504), Elapsed time: 218.53 mins
Enc Loss = 112.07, KL Divergence = 278.39, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 218.60 mins
Memory Use (GB): 3.7045135498046875
Epoch: 70 / 201, Batch: 19 (2560 / 5504), Elapsed time: 218.60 mins
Enc Loss = 112.05, KL Divergence = 274.50, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 218.67 mins
Memory Use (GB): 3.5897140502929688
Epoch: 70 / 201, Batch: 20 (2688 / 5504), Elapsed time: 218.67 mins
Enc Loss = 118.64, KL Divergence = 265.90, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss

Memory Use (GB): 3.6947097778320312
Epoch: 71 / 201, Batch: 5 (768 / 5504), Elapsed time: 220.65 mins
Enc Loss = 108.74, KL Divergence = 267.45, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 220.72 mins
Memory Use (GB): 3.706512451171875
Epoch: 71 / 201, Batch: 6 (896 / 5504), Elapsed time: 220.72 mins
Enc Loss = 111.10, KL Divergence = 275.58, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 220.79 mins
Memory Use (GB): 3.6872711181640625
Epoch: 71 / 201, Batch: 7 (1024 / 5504), Elapsed time: 220.79 mins
Enc Loss = 113.14, KL Divergence = 271.47, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 220.86 mins
Memory Use (GB): 3.7591552734375
Epoch: 71 / 201, Batch: 8 (1152 / 5504), Elapsed time: 220.86 mins
Enc Loss = 116.62, KL Divergence = 272.95, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.

Memory Use (GB): 3.7068748474121094
Epoch: 71 / 201, Batch: 37 (4864 / 5504), Elapsed time: 222.92 mins
Enc Loss = 110.24, KL Divergence = 272.57, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 222.99 mins
Memory Use (GB): 3.786174774169922
Epoch: 71 / 201, Batch: 38 (4992 / 5504), Elapsed time: 222.99 mins
Enc Loss = 116.64, KL Divergence = 280.02, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 223.06 mins
Memory Use (GB): 3.873394012451172
Epoch: 71 / 201, Batch: 39 (5120 / 5504), Elapsed time: 223.07 mins
Enc Loss = 111.74, KL Divergence = 272.92, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 223.14 mins
Memory Use (GB): 3.840526580810547
Epoch: 71 / 201, Batch: 40 (5248 / 5504), Elapsed time: 223.14 mins
Enc Loss = 114.68, KL Divergence = 273.34, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss 

Memory Use (GB): 3.634174346923828
Epoch: 72 / 201, Batch: 25 (3328 / 5504), Elapsed time: 225.12 mins
Enc Loss = 111.78, KL Divergence = 281.79, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 225.19 mins
Memory Use (GB): 3.555156707763672
Epoch: 72 / 201, Batch: 26 (3456 / 5504), Elapsed time: 225.19 mins
Enc Loss = 114.69, KL Divergence = 275.46, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 225.26 mins
Memory Use (GB): 3.7041168212890625
Epoch: 72 / 201, Batch: 27 (3584 / 5504), Elapsed time: 225.27 mins
Enc Loss = 110.32, KL Divergence = 270.28, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss = -999.00, Elapsed time: 225.34 mins
Memory Use (GB): 3.730236053466797
Epoch: 72 / 201, Batch: 28 (3712 / 5504), Elapsed time: 225.34 mins
Enc Loss = 112.65, KL Divergence = 273.10, Reconstruction Loss = 0.01, ll_loss = -999.00, dis_Loss = -999.00, dec_Loss 